## Clustering Analysis Project

## Part-1 ##

**Firstly, installing required libraries**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20201028060147-0000
KERNEL_ID = d73d92f9-a837-4bf8-adc0-fd14403de788


**Secondly, scrape data set from Wikipedia website**

In [2]:
# read the webpage from the wiki
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
#Process and convert html data
data = response.text
soup = BeautifulSoup(data,'html.parser')
wiki_table=soup.find('table')
#develop dataframe
df = pd.read_html(str(wiki_table))[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Thirdly, prepare the dataframe in pursuant of instructions**

In [3]:
# Drop the first column
df.drop(0,inplace=True)
#Rename the columns names
df.columns = ['PostalCode','Borough','Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"


**Cleaning data as ignoring the rows which the "Borough" contains 'Not assigned'**

In [5]:
#Remove "Borough" with 'Not assigned' values
df2=df[df['Borough'].str.contains("Not assigned") == False].reset_index()
df2.head()


,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**Combine the rows into one row (and seperate with comma) which PostalCode and Borough as same value**

In [6]:
df3= df2.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Get the number of rows of dataframe**

In [7]:
df3.shape

(103, 3)

In [8]:
#Save data as'Capstone_part1.csv'
df3.to_csv('Capstone_part1.csv')
print('Successfully Saved!')

Successfully Saved!


## Part - 2 ##

**Firstly, installing required libraries**

In [9]:
import pandas as pd

**Secondly, get the geospatial Coordinates Data from https://cocl.us/Geospatial_data**

In [10]:
# get the data and create the dataframe
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Adjust the column name same as the first dataset**

In [11]:
df_geo = df_geo.rename(columns = {'Postal Code':'PostalCode'}) 
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#We upload the first dataset where we save it as csv file in part 1
df3=pd.read_csv('Capstone_part1.csv')
df3.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood
0,0,M1B,Scarborough,"Malvern, Rouge"
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae


**Merge two dataframes based on the 'PostalCode' column**

In [13]:
df3 = pd.merge(df3, df_geo, on = 'PostalCode')
df3.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
df3.shape

(103, 6)

In [15]:
#Save data as'Capstone_part2.csv'
df3.to_csv('Capstone_part2.csv', index=False)
print('Successfully Saved!')

Successfully Saved!


## Part - 3 ##

**Firstly, installing required libraries**

In [16]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

**Secondly, Upload the first dataset**

In [17]:
#We upload the first dataset where we save it as csv file in part 2
df3=pd.read_csv('Capstone_part2.csv')
df3.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Thirdly, prepare the dataframe in pursuant of instructions**

In [18]:
#We work with only "Borough" which contain the word Toronto 
df4=df3[df3['Borough'].str.contains('Toronto')]
df5=df4.reset_index(drop=True)
df5.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [19]:
df5.shape

(39, 6)

**For clustering the data, we can use the Borough as Label**

In [20]:
# Learn the different value of Borough 
df5['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [21]:
# Create a new column as Label and get the date from 'Borough' as integer
df5['Label']=df5['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto'],value=[1,2,3,4],inplace=False)
df5.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude,Label
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4
2,42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,4
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,4
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2


**Fourthly, visualize the data**

Learn the coordinates of Toronto

In [22]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Preparing to create the clustering map of Toronto**

In [23]:
#for set the cluster number as label number
kclusters=len(df5.Label.unique())

# create map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df5['Latitude'], df5['Longitude'], df5['Label']):
    label = folium.Popup(str(df5['Borough']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

**Display the neighborhood map of Toronto with 4 clustering**

In [24]:
map_toronto

## Thank You ##